In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
import sys,os
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from tianmoucv.data import TianmoucDataReader
import matplotlib.pyplot as plt


train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]

val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]

print('---------------------------------------------------')
for sampleset in traindata:
    for e in os.listdir(sampleset):
        print(e)
print('---------------------------------------------------')
for sampleset in valdata:
    for e in os.listdir(sampleset):
        print(e)
        
alldata = valdata + traindata

In [ ]:
%autoreload
import time
from tianmoucv.isp import lyncam_raw_comp,demosaicing_npy,fourdirection2xy
from tianmoucv.proc.features.diff import HarrisCorner,sift,hog
from tianmoucv.proc.tracking.feature_tracker import feature_matching,mini_l2_cost_matching,align_images
from tianmoucv.proc.reconstruct.integration import grayReconstructor

import cv2

fl_aim = []
kp_aim = []

gap  = 1

#目前的策略比较粗暴，间隔一个固定的帧就直接重新找特征点，防止积累误差
fix_update_frame = 750//gap
tracking_count = 0

imlist = []
time_begin = time.time()
history = dict([])

key= 'outdoor_bridge_3'
startID = 80
endID = startID + 10
dataset = TianmoucDataReader(alldata,MAXLEN=500*1,matchkey=key,speedUpRate=1)

with torch.no_grad():
    for index in range(len(dataset)):
        if index<startID:
            continue
        if index>endID:
            break
        else:
            sample = dataset[index]
            F0 = sample['F0']
            F1 = sample['F1']
            tsdiff = sample['rawDiff']
            threshed_tsdiff = tsdiff[:,...].permute(1,2,3,0)
            
            for t in range(0,threshed_tsdiff.shape[0],gap):
                
                tsdiff = torch.Tensor(sample['rawDiff'])
                hdr_show = tsdiff[:,t,...].permute(1,2,0).numpy().copy()/255
                hdr_show = cv2.resize(hdr_show,(640,320))
                SD = threshed_tsdiff[t,...,1:]
                TD = threshed_tsdiff[t,...,0]
                Ix1,Iy1= fourdirection2xy(SD)
                Ix1 = torch.tensor(Ix1).float()
                Iy1 = torch.tensor(Iy1).float()
                Ix1[abs(Ix1)<5]=0
                Iy1[abs(Iy1)<5]=0
                Ix1/=255.0
                Iy1/=255.0
                
                featureList1 = []
                kp1=[]
                #第1步：计算两张图对应Harris角点检测
                startT = time.time()
                
                #TODO:改成自适应阈值，控制特征点数量和密度
                idmap,R = HarrisCorner(Ix1,Iy1,k=0.01,th=0.04)
                endT = time.time()

                for i in range(idmap.shape[0]):
                    for j in range(idmap.shape[1]):
                        if idmap[i,j]>0:
                            kp1.append([i,j])
                #第2步：计算两张图对应fp list的特征描述子
                startT = time.time()
                kp1,featureList1 = sift(Ix1,Iy1,kp1)
                endT = time.time()
                kp1 = [ (p[0]*2,p[1]*4) for p in kp1 ]
                fl1 = torch.stack(featureList1,dim=0).view(len(kp1),-1)
                
                #第3步：更新待追踪特征点列表
                if tracking_count % fix_update_frame == 0 or len(fl_aim)==0:
                    print('update tracking target')
                    kp_aim = kp1
                    fl_aim = fl1
                    history = dict([])
                    for i in range(len(kp_aim)):
                        history[i] = [ kp1[i] ]
                else:
                    if len(fl1)>0:
                        matches = feature_matching(fl_aim,fl1, ratio=0.75)
                        #print('matched kp:',len(matches))
                        
                        #只要匹配上，就更新待追踪点坐标和对应的特征描述子，以免场景变化过大影响追踪
                        for m in matches:
                            src_pts = kp_aim[m[0].queryIdx]
                            dst_pts = kp1[m[0].trainIdx]
                            dist = (src_pts[0]-dst_pts[0])**2 + (src_pts[1]-dst_pts[1])**2
                            
                            #750fps,过远的匹配认为是错的，这个值可以改
                            if dist < 400:
                                history[m[0].queryIdx].append(kp1[m[0].trainIdx])
                                kp_aim[m[0].queryIdx] = kp1[m[0].trainIdx]
                                fl_aim[m[0].queryIdx,:] = fl1[m[0].trainIdx,:]
                                
                        #绘制追踪结果
                        F_show = F1.copy()
                        for k in history:
                            traj = history[k]
                            y2, x2 = (None,None)
                            for kp in traj:
                                y1 , x1 = int(kp[0]),int(kp[1])
                                #cv2.circle(F_show,(x1,y1),2,(0,0.2,0.2))
                                if not x2 is None:
                                    cv2.line(F_show,(x1,y1),(x2,y2),(0,1,0))
                                    cv2.line(hdr_show,(x1,y1),(x2,y2),(0,1,0))
                                y2 = y1
                                x2 = x1
                            cv2.circle(F_show,(x2,y2),2,(0,1,1))
                            cv2.circle(hdr_show,(x2,y2),2,(0,1,1))
                            
                        imshow = np.concatenate([F_show,hdr_show],axis=0)
                        imlist.append(imshow)
                    else:
                        print('no useable new feature')
                tracking_count += 1
                
            if index%1 ==0:
                time_10 = time.time()
                print('avg fps:',1/(time_10-time_begin))
                time_begin = time.time()
                plt.figure(figsize=(12,6))
                plt.subplot(1,2,1)
                plt.imshow(F_show)
                plt.subplot(1,2,2)
                plt.imshow(hdr_show)
                plt.show()
            


In [ ]:
from tianmoucv.algorithm import white_balance

def images_to_video(frame_list,name,Val_size=(512,256),Flip=False):
    fps = 30     
    size = (Val_size[0], Val_size[1]*2) # 需要转为视频的图片的尺寸
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for frame in frame_list:
        frame = (frame-np.min(frame))/(np.max(frame)-np.min(frame)) * 255
        frame2 = frame.astype(np.uint8)
        out.write(frame2)
    out.release()
    
images_to_video(imlist,'./'+key+'.mp4',Val_size=(640,320),Flip=False)

# 